In [3]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem

# Load drugs (SMILES)
with open("ligands.txt") as f:
    drugs = [line.strip() for line in f]

# Load proteins (sequences)
with open("proteins.txt") as f:
    proteins = [line.strip() for line in f]

# Load affinity matrix (usually in .npy or .pkl format in Davis repo)
Y = np.loadtxt("affinity.txt")   # shape = (num_drugs, num_proteins)


In [4]:
print("Affinity matrix shape:", Y.shape)

Affinity matrix shape: (68, 442)


In [5]:
print(Y[:5, :5])

[[4.3e+01 1.0e+04 1.0e+04 1.0e+04 1.0e+04]
 [1.0e+04 1.0e+04 1.0e+04 1.0e+04 1.0e+04]
 [1.0e+04 7.5e+01 1.9e+00 1.3e+01 7.7e-01]
 [1.0e+04 1.0e+04 1.0e+04 1.0e+04 1.0e+04]
 [1.0e+04 4.2e+02 2.9e+03 7.5e+02 5.8e+02]]


In [8]:
import json
import numpy as np
import pandas as pd

# Load proteins.json (actually your proteins.txt)
with open("proteins.txt") as f:
    proteins_dict = json.load(f)

# Extract sequences and IDs
proteins = list(proteins_dict.values())
protein_ids = list(proteins_dict.keys())

print(f"✅ Loaded {len(proteins)} proteins")
print("Example:", protein_ids[0], "->", proteins[0][:30] + "...")

# One-hot encoding of protein sequences (simple example)
# Here we just pad/truncate to 1000 chars
MAX_SEQ_LEN = 1000
AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY"  # 20 standard aa
aa_to_idx = {aa: i for i, aa in enumerate(AMINO_ACIDS)}

def encode_protein(seq):
    arr = np.zeros((MAX_SEQ_LEN, len(AMINO_ACIDS)), dtype=int)
    for i, aa in enumerate(seq[:MAX_SEQ_LEN]):
        if aa in aa_to_idx:
            arr[i, aa_to_idx[aa]] = 1
    return arr.flatten()

X_protein = np.array([encode_protein(seq) for seq in proteins])

print("Proteins shape:", X_protein.shape)


✅ Loaded 442 proteins
Example: AAK1 -> MKKFFDSRREQGGSGLGSGSSGGGGSTSGL...
Proteins shape: (442, 20000)


In [10]:
# -------------------------------
# DeepDTA Preprocessing Full Code
# -------------------------------
import json
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

# -------------------------------
# Step 1: Load ligands (drugs)
# -------------------------------
with open("ligands.txt") as f:
    ligands_dict = json.load(f)   # full JSON

drug_ids = list(ligands_dict.keys())
drugs = list(ligands_dict.values())
print(f"✅ Loaded {len(drugs)} ligands")
print("Example:", drug_ids[0], "->", drugs[0][:50] + "...")

# -------------------------------
# Step 2: Convert SMILES → Morgan Fingerprints
# -------------------------------
def smiles_to_fp(smiles, radius=2, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)
        arr = np.zeros((nBits,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, arr)
        return arr
    else:
        print("⚠️ Failed to parse:", smiles)
        return np.zeros((nBits,), dtype=int)

X_drug = np.array([smiles_to_fp(smi) for smi in drugs])
print("Drugs shape:", X_drug.shape)  # should be (68, 1024)

# -------------------------------
# Step 3: Load proteins
# -------------------------------
with open("proteins.txt") as f:
    proteins_dict = json.load(f)

protein_ids = list(proteins_dict.keys())
proteins = list(proteins_dict.values())
print(f"✅ Loaded {len(proteins)} proteins")
print("Example:", protein_ids[0], "->", proteins[0][:50] + "...")

# -------------------------------
# Step 4: Convert protein sequences → one-hot (flattened)
# -------------------------------
MAX_SEQ_LEN = 1000
AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY"
aa_to_idx = {aa: i for i, aa in enumerate(AMINO_ACIDS)}

def encode_protein(seq):
    arr = np.zeros((MAX_SEQ_LEN, len(AMINO_ACIDS)), dtype=int)
    for i, aa in enumerate(seq[:MAX_SEQ_LEN]):
        if aa in aa_to_idx:
            arr[i, aa_to_idx[aa]] = 1
    return arr.flatten()

X_protein = np.array([encode_protein(seq) for seq in proteins])
print("Proteins shape:", X_protein.shape)  # (442, 20000)

# -------------------------------
# Step 5: Load affinity matrix
# -------------------------------
Y = np.loadtxt("affinity.txt")  # shape = (68, 442)
print("Affinity matrix shape:", Y.shape)

# -------------------------------
# Step 6: Create (drug, protein, affinity) triplets
# -------------------------------
data = []
for i in range(X_drug.shape[0]):        # 68 drugs
    for j in range(X_protein.shape[0]): # 442 proteins
        data.append((X_drug[i], X_protein[j], Y[i,j]))

X_drug_all = np.array([d for d, p, y in data])
X_protein_all = np.array([p for d, p, y in data])
y_all = np.array([y for d, p, y in data]).reshape(-1, 1)

print("Total samples created:", X_drug_all.shape[0])
print("Drugs shape:", X_drug_all.shape)
print("Proteins shape:", X_protein_all.shape)
print("Labels shape:", y_all.shape)

# -------------------------------
# Step 7: Save preprocessed data
# -------------------------------
np.save("X_drug.npy", X_drug_all)
np.save("X_protein.npy", X_protein_all)
np.save("y.npy", y_all)
print("🎉 Preprocessing complete! Files saved: X_drug.npy, X_protein.npy, y.npy")


✅ Loaded 68 ligands
Example: 11314340 -> CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)...
Drugs shape: (68, 1024)
✅ Loaded 442 proteins
Example: AAK1 -> MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDE...
Proteins shape: (442, 20000)
Affinity matrix shape: (68, 442)


[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerator
[10:56:11] DEPRECATION WARNING: please use MorganGenerat

Total samples created: 30056
Drugs shape: (30056, 1024)
Proteins shape: (30056, 20000)
Labels shape: (30056, 1)
🎉 Preprocessing complete! Files saved: X_drug.npy, X_protein.npy, y.npy


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [12]:
X_drug = np.load("X_drug.npy")        # shape: (30056, 1024)
X_protein = np.load("X_protein.npy")  # shape: (30056, 20000)
y = np.load("y.npy")                  # shape: (30056, 1)

# Standardize labels (optional, helps training)
scaler = StandardScaler()
y = scaler.fit_transform(y)

In [13]:
X_drug_train, X_drug_test, X_protein_train, X_protein_test, y_train, y_test = train_test_split(
    X_drug, X_protein, y, test_size=0.1, random_state=42
)

In [14]:
class DTADataset(Dataset):
    def __init__(self, X_drug, X_protein, y):
        self.X_drug = torch.tensor(X_drug, dtype=torch.float32)
        self.X_protein = torch.tensor(X_protein, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X_drug[idx], self.X_protein[idx], self.y[idx]

train_dataset = DTADataset(X_drug_train, X_protein_train, y_train)
test_dataset = DTADataset(X_drug_test, X_protein_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [19]:
class DeepDTA(nn.Module):
    def __init__(self):
        super(DeepDTA, self).__init__()
        
        # Drug branch
        self.drug_cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4),
            nn.Conv1d(32, 64, kernel_size=8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4)
        )
        
        # Protein branch
        self.protein_cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4),
            nn.Conv1d(32, 64, kernel_size=8),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4)
        )
        
        # Dummy input to compute flattened size
        self._to_linear = None
        self._compute_flattened_size()
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(self._to_linear, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
    
    def _compute_flattened_size(self):
        # create dummy tensors
        x_drug = torch.zeros(1, 1, 1024)
        x_protein = torch.zeros(1, 1, 20000)
        
        drug_feat = self.drug_cnn(x_drug)
        protein_feat = self.protein_cnn(x_protein)
        
        self._to_linear = drug_feat.view(1, -1).shape[1] + protein_feat.view(1, -1).shape[1]
    
    def forward(self, drug, protein):
        drug = drug.unsqueeze(1)       # (batch, 1, 1024)
        protein = protein.unsqueeze(1) # (batch, 1, 20000)
        
        drug_feat = self.drug_cnn(drug)
        drug_feat = drug_feat.view(drug_feat.size(0), -1)
        
        protein_feat = self.protein_cnn(protein)
        protein_feat = protein_feat.view(protein_feat.size(0), -1)
        
        x = torch.cat([drug_feat, protein_feat], dim=1)
        out = self.fc(x)
        return out


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepDTA().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 10

In [21]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_drug, batch_protein, batch_y in train_loader:
        batch_drug = batch_drug.to(device)
        batch_protein = batch_protein.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_drug, batch_protein)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * batch_drug.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/10, Loss: 0.8652
Epoch 2/10, Loss: 0.5845
Epoch 3/10, Loss: 0.5306
Epoch 4/10, Loss: 0.4682
Epoch 5/10, Loss: 0.4148
Epoch 6/10, Loss: 0.3735
Epoch 7/10, Loss: 0.3357
Epoch 8/10, Loss: 0.2989
Epoch 9/10, Loss: 0.2603
Epoch 10/10, Loss: 0.2272


In [22]:
model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch_drug, batch_protein, batch_y in test_loader:
        batch_drug = batch_drug.to(device)
        batch_protein = batch_protein.to(device)
        batch_y = batch_y.to(device)
        
        outputs = model(batch_drug, batch_protein)
        preds.append(outputs.cpu().numpy())
        trues.append(batch_y.cpu().numpy())

preds = np.vstack(preds)
trues = np.vstack(trues)

# Inverse scaling
preds = scaler.inverse_transform(preds)
trues = scaler.inverse_transform(trues)

# Compute RMSE
rmse = np.sqrt(np.mean((preds - trues)**2))
print(f"Test RMSE: {rmse:.4f}")

Test RMSE: 2623.6458


In [23]:
import numpy as np

y = np.load("y.npy")  # original labels
y = np.log(y + 1e-8)  # add small epsilon to avoid log(0)


In [26]:
y_pred_original = np.exp(y)
print(y_pred_original)

[[   43.00000001]
 [10000.00000001]
 [10000.00000001]
 ...
 [ 1900.00000001]
 [ 4400.00000001]
 [10000.00000001]]
